In [1]:
#import the necessary packages and functions
import os
from Crypto.Cipher import AES
from Crypto import Random
import random as rand
import subprocess
import matplotlib.pyplot as plt
import time
%matplotlib inline

In [2]:
#creating dictionary containing english letter frequencies
pos = {'a':8167, 'b':1492, 'c':2782, 'd':4253, 'e':12702, 'f':2228, 'g':2015, 'h':6094, 'i':6966,
       'j':153, 'k':772, 'l':4025, 'm':2406, 'n':6749, 'o':7507, 'p':1929, 'q':95, 'r':5987,
       's':6327, 't':9056, 'u':2758, 'v':978, 'w':2360, 'x':150, 'y':1974, 'z':74}

In [3]:
#initialize plaintext
plaintext = ""

#run loop to sample 16 letters and append them to plaintext
for i in range(16):
    plaintext += rand.choice([x for x in pos for y in range(pos[x])])

print("plaintext:\n", plaintext)

#convert plaintext to hexadecimal then bit string
plaintext_hex = plaintext.encode('cp037').hex()
plaintext_bit = bin(int(plaintext_hex, 16))[2:]

print("\nplaintext_hex:\n", plaintext_hex,"\nlength:\n",len(plaintext_hex))
print("\nplaintext_bit:\n", plaintext_bit,"\nlength:\n",len(plaintext_bit))

plaintext:
 vebesrscaendcsos

plaintext_hex:
 a5858285a299a2838185958483a296a2 
length:
 32

plaintext_bit:
 10100101100001011000001010000101101000101001100110100010100000111000000110000101100101011000010010000011101000101001011010100010 
length:
 128


In [4]:
#generate random key using os.urandom() PRNG
key = os.urandom(16)

print("key:\n", key)

#convert key to hexadecimal then bit string
key_hex = key.hex()
key_bit = bin(int(key_hex, 16))[2:]

print("\nkey_hex:\n", key_hex,"\nlength:\n",len(key_hex))
print("\nkey_bit:\n", key_bit,"\nlength:\n",len(key_bit))

key:
 b'\x8b\xf1\xf1\xd5h\xcej^\x0e8\xf7a\x936\x84\xcd'

key_hex:
 8bf1f1d568ce6a5e0e38f761933684cd 
length:
 32

key_bit:
 10001011111100011111000111010101011010001100111001101010010111100000111000111000111101110110000110010011001101101000010011001101 
length:
 128


In [5]:
#generate AES cipher object with 128-bit key
cipher = AES.new(key)

#encrypt plaintext with AES cipher object
ciphertext = cipher.encrypt(plaintext)

print("ciphertext:\n", ciphertext)

#convert key to hexadecimal then bit string
ciphertext_hex = ciphertext.hex()
ciphertext_bit = bin(int(ciphertext_hex, 16))[2:]

print("\nkey_hex:\n", ciphertext_hex,"\nlength:\n",len(ciphertext_hex))
print("\nkey_bit:\n", ciphertext_bit,"\nlength:\n",len(ciphertext_bit))

ciphertext:
 b'\xe1L\xc2}@\xb5\x83p\xe4\xb2\xabd\x95\xbcO\x96'

key_hex:
 e14cc27d40b58370e4b2ab6495bc4f96 
length:
 32

key_bit:
 11100001010011001100001001111101010000001011010110000011011100001110010010110010101010110110010010010101101111000100111110010110 
length:
 128


In [6]:
#!/usr/bin/env python

import numpy as np
import scipy.special as spc
import scipy.fftpack as sff
import scipy.stats as sst
import functools

def sumi(x): return 2 * x - 1
def su(x, y): return x + y
def sus(x): return (x - 0.5) ** 2
def sq(x): return int(x) ** 2
def logo(x): return x * np.log(x)

def pr(u, x):
    if u == 0:
        out=1.0 * np.exp(-x)
    else:
        out=1.0 * x * np.exp(2*-x) * (2**-u) * spc.hyp1f1(u + 1, 2, x)
    return out

def stringpart(binin, num):
    blocks = [binin[xs * num:num + xs * num:] for xs in range(int(len(binin) / num))]
    return blocks

def randgen(num):
    '''Spits out a stream of random numbers like '1001001' with the length num'''

    rn = open('/dev/urandom', 'r')
    random_chars = rn.read(num / 2)
    stream = ''
    for char in random_chars:
        c = ord(char)
        for i in range(0, 2):
            stream += str(c >> i & 1)
    return stream

<h3>Randomness Test #1: Frequency (Monobit) Test</h3>
Input: bit string

In [7]:
def monobitfrequencytest(binin):
    ''' The focus of the test is the proportion of zeroes and ones for the entire sequence. The purpose of this 
    test is to determine whether that number of ones and zeros in a sequence are approximately the same as would 
    be expected for a truly random sequence. The test assesses the closeness of the fraction of ones to 1/2, that 
    is, the number of ones and zeroes in a sequence should be about the same.'''
    ss = [int(el) for el in binin]
    sc = list(map(sumi, ss))
    sn = functools.reduce(su, sc)
    sobs = np.abs(sn) / np.sqrt(len(binin))
    pval = spc.erfc(sobs / np.sqrt(2))
    return pval

In [8]:
monobitfrequencytest(ciphertext_bit)

0.7236736098317631

<h3>Randomness Test #2: Frequency Test within a Block</h3>
Input: bit string, block length

In [9]:
def blockfrequencytest(binin, nu=128):
    ''' The focus of the test is the proportion of zeroes and ones within M-bit blocks. The purpose of this test 
    is to determine whether the frequency of ones is an M-bit block is approximately M/2.'''
    ss = [int(el) for el in binin]
    tt = [1.0 * sum(ss[xs * nu:nu + xs * nu:]) / nu for xs in range(int(len(ss) / nu))]
    uu = list(map(sus, tt))
    chisqr = 4 * nu * functools.reduce(su, uu)
    pval = spc.gammaincc(len(tt) / 2.0, chisqr / 2.0)
    return pval

In [10]:
blockfrequencytest(ciphertext_bit, nu=128)

0.72367360983176288

<h3>Randomness Test #3: Runs Test</h3>
Input: bit string

In [11]:
def runstest(binin):
    ''' The focus of this test is the total number of zero and one runs in the entire sequence, where a run is an 
    uninterrupted sequence of identical bits. A run of length k means that a run consists of exactly k identical 
    bits and is bounded before and after with a bit of the opposite value. The purpose of the runs test is to 
    determine whether the number of runs of ones and zeros of various lengths is as expected for a random sequence. 
    In particular, this test determines whether the oscillation between such substrings is too fast or too slow.''' 
    ss = [int(el) for el in binin]
    n = len(binin)
    pi = 1.0 * functools.reduce(su, ss) / n
    vobs = len(binin.replace('0', ' ').split()) + len(binin.replace('1' , ' ').split())
    pval = spc.erfc(abs(vobs-2*n*pi*(1-pi)) / (2 * pi * (1 - pi) * np.sqrt(2*n)))
    return pval

In [12]:
runstest(ciphertext_bit)

0.28338045965109182

<h3>Randomness Test #4: Test for Longest Run of Ones in a Block</h3>
Input: bit string  
NOTE: 3 different functions depending on bit string length

In [13]:
def longestrunones8(binin):
    ''' The focus of the test is the longest run of ones within M-bit blocks. The purpose of this test is to 
    determine whether the length of the longest run of ones within the tested sequence is consistent with the 
    length of the longest run of ones that would be expected in a random sequence. Note that an irregularity in 
    the expected length of the longest run of ones implies that there is also an irregularity in the expected 
    length of the longest run of zeroes. Long runs of zeroes were not evaluated separately due to a concern about 
    statistical independence among the tests.'''
    if len(binin) > 127:
        m = 8
        k = 3
        pik = [0.2148, 0.3672, 0.2305, 0.1875]
        blocks = [binin[xs*m:m+xs*m:] for xs in range(int(len(binin) / m))]
        n = len(blocks)
        counts1 = [xs+'01' for xs in blocks] # append the string 01 to guarantee the length of 1
        counts = [xs.replace('0',' ').split() for xs in counts1] # split into all parts
        counts2 = [list(map(len, xx)) for xx in counts]
        counts4 = [(4 if xx > 4 else xx) for xx in list(map(max,counts2))]
        freqs = [counts4.count(spi) for spi in [1, 2, 3, 4]]
        chisqr1 = [(freqs[xx]-n*pik[xx])**2/(n*pik[xx]) for xx in range(4)]
        chisqr = functools.reduce(su, chisqr1)
        pval = spc.gammaincc(k / 2.0, chisqr / 2.0)
    else:
        print('longestrunones8 failed, too few bits:', len(binin))
        pval = 0
    return pval

def longestrunones128(binin):  # not well tested yet
    if len(binin) > 6271:
        m = 128
        k = 5
        n = len(binin)
        pik = [ 0.1174, 0.2430, 0.2493, 0.1752, 0.1027, 0.1124 ]
        blocks = [binin[xs * m:m + xs * m:] for xs in range(int(len(binin) / m))]
        n = len(blocks)
        counts = [xs.replace('0', ' ').split() for xs in blocks]
        counts2 = [list(map(len, xx)) for xx in counts]
        counts3 = [(4 if xx < 5 else xx) for xx in list(map(max, counts2))]
        counts4 = [(9 if xx > 9 else xx) for xx in counts3]
        freqs = [counts4.count(spi) for spi in [4, 5, 6, 7, 8, 9]]
        chisqr1 = [(freqs[xx] - n * pik[xx]) ** 2 / (n * pik[xx]) for xx in range(len(freqs))]
        chisqr = functools.reduce(su, chisqr1)
        pval = spc.gammaincc(k / 2.0, chisqr / 2.0)
    else:
        print('longestrunones128 failed, too few bits:', len(binin))
        pval = 0
    return pval

def longestrunones10000(binin):  # not well tested yet
    ''' The focus of the test is the longest run of ones within M-bit blocks. The purpose of this test is to 
    determine whether the length of the longest run of ones within the tested sequence is consistent with the 
    length of the longest run of ones that would be expected in a random sequence. Note that an irregularity in 
    the expected length of the longest run of ones implies that there is also an irregularity in the expected 
    length of the longest run of zeroes. Long runs of zeroes were not evaluated separately due to a concern about 
    statistical independence among the tests.'''
    if len(binin) > 749999:
        m = 10000
        k = 6
        pik = [0.0882, 0.2092, 0.2483, 0.1933, 0.1208, 0.0675, 0.0727]
        blocks = [binin[xs * m:m + xs * m:] for xs in range(int(len(binin) / m))]
        n = len(blocks)
        counts = [xs.replace('0', ' ').split() for xs in blocks]
        counts2 = [list(map(len, xx)) for xx in counts]
        counts3 = [(10 if xx < 11 else xx) for xx in list(map(max, counts2))]
        counts4 = [(16 if xx > 16 else xx) for xx in counts3]
        freqs = [counts4.count(spi) for spi in [10,11,12,13,14,15,16]]
        chisqr1 = [(freqs[xx] - n * pik[xx]) ** 2 / (n * pik[xx]) for xx in range(len(freqs))]
        chisqr = functools.reduce(su, chisqr1)
        pval = spc.gammaincc(k / 2.0, chisqr / 2.0)
    else:
        print('longestrunones10000 failed, too few bits:', len(binin))
        pval = 0
    return pval

In [14]:
longestrunones8(ciphertext_bit)

0.682855154252068

In [15]:
longestrunones128(ciphertext_bit*49)

3.6490259082223474e-31

In [16]:
longestrunones10000(ciphertext_bit*5860)

3.2746412530841405e-164

<h3>Randomness Test #5: Binary Matrix Rank Test</h3>
Input: bit string, number of rows in each matrix (m), number of columns in each matrix (q)

In [17]:
def mrank(matrix): # matrix rank as defined in the NIST specification
    m=len(matrix)
    leni=len(matrix[0])
    def proc(mat):
        for i in range(m):
            if mat[i][i]==0:
                for j in range(i+1,m):
                    if mat[j][i]==1:
                        mat[j],mat[i]=mat[i],mat[j]
                        break
            if mat[i][i]==1:
                for j in range(i+1,m):
                    if mat[j][i]==1: mat[j]=[mat[i][x]^mat[j][x] for x in range(leni)]
        return mat
    maa=proc(matrix)
    maa.reverse()
    mu=[i[::-1] for i in maa]
    muu=proc(mu)
    ra=np.sum(np.sign([xx.sum() for xx in np.array(mu)]))
    return ra

def binarymatrixranktest(binin,m=32,q=32):
    ''' The focus of the test is the rank of disjoint sub-matrices of the entire sequence. The purpose of this test is to check for linear dependence among fixed length substrings of the original sequence.'''
    p1 = 1.0
    for x in range(1,50): p1*=1-(1.0/(2**x))
    p2 = 2*p1
    p3 = 1-p1-p2;
    n=len(binin)
    u=[int(el) for el in binin] # the input string as numbers, to generate the dot product 
    f1a = [u[xs*m:xs*m+m:] for xs in range(int(n/m))]
    n=len(f1a)
    f2a = [f1a[xs*q:xs*q+q:] for xs in range(int(n/q))]
    r=list(map(mrank,f2a))
    n=len(r)
    fm=r.count(m);
    fm1=r.count(m-1);
    chisqr=((fm-p1*n)**2)/(p1*n)+((fm1-p2*n)**2)/(p2*n)+((n-fm-fm1-p3*n)**2)/(p3*n);
    pval=np.exp(-0.5*chisqr)
    return pval

In [18]:
binarymatrixranktest((ciphertext_bit*2),m=16,q=16)

0.039104615860550376

<h3>Randomness Test #6: Spectral Test</h3>
Input: bit string

In [19]:
def spectraltest(binin):
    '''The focus of this test is the peak heights in the discrete Fast Fourier Transform. The purpose of this test 
    is to detect periodic features (i.e., repetitive patterns that are near each other) in the tested sequence that 
    would indicate a deviation from the assumption of randomness. '''
    n = len(binin)
    ss = [int(el) for el in binin]
    sc = list(map(sumi, ss))
    ft = sff.fft(sc)
    af = abs(ft)[1:int(n/2+1):]
    t = np.sqrt(np.log(1/0.05)*n)
    n0 = 0.95*n/2
    n1 = len(np.where(af<t)[0])
    d = (n1 - n0)/np.sqrt(n*0.95*0.05/4)
    pval = spc.erfc(abs(d)/np.sqrt(2))
    return pval

In [20]:
spectraltest(ciphertext_bit)

0.33039004884879331

<h3>Randomness Test #7: Non-overlapping Template Matching Test</h3>
Input: bit string, m-bit template to be matched(mat), number of independent blocks(num)

In [21]:
def nonoverlappingtemplatematchingtest(binin, mat="000000001", num=8):
    ''' The focus of this test is the number of occurrences of pre-defined target substrings. The purpose of this 
    test is to reject sequences that exhibit too many occurrences of a given non-periodic (aperiodic) pattern. For 
    this test and for the Overlapping Template Matching test, an m-bit window is used to search for a specific 
    m-bit pattern. If the pattern is not found, the window slides one bit position. For this test, when the pattern 
    is found, the window is reset to the bit after the found pattern, and the search resumes.'''
    n = len(binin)
    m = len(mat)
    M = int(n/num)
    blocks = [binin[xs*M:M+xs*M:] for xs in range(int(n/M))]
    counts = [xx.count(mat) for xx in blocks]
    avg = 1.0 * (M-m+1)/2 ** m
    var = M*(2**-m -(2*m-1)*2**(-2*m))
    chisqr = functools.reduce(su, [(xs - avg) ** 2 for xs in counts]) / var
    pval = spc.gammaincc(1.0 * len(blocks) / 2, chisqr / 2)
    return pval

In [22]:
nonoverlappingtemplatematchingtest(ciphertext_bit, mat="000000001", num=8)

0.99999995567755462

<h3>Randomness Test #8: Overlapping Template Matching Test</h3>
Input: bit string, m-bit template to be matched(mat), number of independent blocks(num), degrees of freedom(numi)

In [23]:
def occurances(string, sub):
    count=start=0
    while True:
        start=string.find(sub,start)+1
        if start>0:
            count+=1
        else:
            return count

def overlappingtemplatematchingtest(binin,mat="111111111",num=1032,numi=5):
    ''' The focus of this test is the number of pre-defined target substrings. The purpose of this test is to 
    reject sequences that show deviations from the expected number of runs of ones of a given length. Note that 
    when there is a deviation from the expected number of ones of a given length, there is also a deviation in the 
    runs of zeroes. Runs of zeroes were not evaluated separately due to a concern about statistical independence 
    among the tests. For this test and for the Non-overlapping Template Matching test, an m-bit window is used to 
    search for a specific m-bit pattern. If the pattern is not found, the window slides one bit position. For this 
    test, when the pattern is found, the window again slides one bit, and the search is resumed.'''
    n = len(binin)
    bign = int(n / num)
    m = len(mat)
    lamda = 1.0 * (num - m + 1) / 2 ** m
    eta = 0.5 * lamda
    pi = [pr(i, eta) for i in range(numi)]
    pi.append(1 - functools.reduce(su, pi))
    v = [0 for x in range(numi + 1)]
    blocks = stringpart(binin, num)
    blocklen = len(blocks[0])
    counts = [occurances(i,mat) for i in blocks]
    counts2 = [(numi if xx > numi else xx) for xx in counts]
    for i in counts2: v[i] = v[i] + 1
    chisqr = functools.reduce(su, [(v[i]-bign*pi[i])** 2 / (bign*pi[i]) for i in range(numi + 1)])
    pval = spc.gammaincc(0.5*numi, 0.5*chisqr)
    return pval

In [24]:
overlappingtemplatematchingtest(ciphertext_bit*20,mat="111111111",num=1032,numi=5)

0.63300733691853539

<h3>Randomness Test #9: Maurer's "Universal Statistical" Test</h3>
Input: bit string, length of each block(l), number of blocks in intialization sequence(q)

In [25]:
def maurersuniversalstatistictest(binin,l=2,q=40):
    ''' The focus of this test is the number of bits between matching patterns. The purpose of the test is to 
    detect whether or not the sequence can be significantly compressed without loss of information. An overly 
    compressible sequence is considered to be non-random.'''
    ru = [
        [0.7326495, 0.690],
        [1.5374383, 1.338],
        [2.4016068, 1.901],
        [3.3112247, 2.358],
        [4.2534266, 2.705],
        [5.2177052, 2.954],
        [6.1962507, 3.125],
        [7.1836656, 3.238],
        [8.1764248, 3.311],
        [9.1723243, 3.356],
        [10.170032, 3.384],
        [11.168765, 3.401],
        [12.168070, 3.410],
        [13.167693, 3.416],
        [14.167488, 3.419],
        [15.167379, 3.421],
        ]
    blocks = [int(li, 2) + 1 for li in stringpart(binin, l)]
    k = len(blocks) - q
    states = [0 for x in range(int(2**l))]
    for x in range(q):
        states[blocks[x]-1]=x+1
    sumi=0.0
    for x in range(q,len(blocks)):
        sumi+=np.log2((x+1)-states[blocks[x]-1])
        states[blocks[x]-1] = x+1
    fn = sumi / k
    c=0.7-(0.8/l)+(4+(32.0/l))*((k**(-3.0/l))/15)
    sigma=c*np.sqrt((ru[l-1][1])/k)
    pval = spc.erfc(abs(fn-ru[l-1][0]) / (np.sqrt(2)*sigma))
    return pval

In [26]:
maurersuniversalstatistictest(ciphertext_bit*10,l=2,q=40)

0.00046160163083415874

<h3>Randomness Test #10: Linear Complexity Test</h3>
Input: bit string, length of each block(m)

In [27]:
def lincomplex(binin):
    lenn=len(binin)
    c=b=np.zeros(lenn)
    c[0]=b[0]=1
    l=0
    m=-1
    n=0
    u=[int(el) for el in binin] # the input string as numbers, to generate the dot product
    p=99
    while n<lenn:
        v=u[(n-l):n] # was n-l..n-1
        v.reverse()
        cc=c[1:l+1] # was 2..l+1
        d=(u[n]+np.dot(v,cc))%2
        if d==1:
            tmp=c
            p=np.zeros(lenn)
            for i in range(0,l): # was 1..l+1
                 if b[i]==1: 
                     p[i+n-m]=1
            c=(c+p)%2;
            if l<=0.5*n: # was if 2l <= n
                 l=n+1-l
                 m=n
                 b=tmp
        n+=1
    return l

# test 2.10
def linearcomplexitytest(binin,m=500):
    ''' The focus of this test is the length of a generating feedback register. The purpose of this test is to 
    determine whether or not the sequence is complex enough to be considered random. Random sequences are 
    characterized by a longer feedback register. A short feedback register implies non-randomness.'''
    k = 6
    pi = [0.01047, 0.03125, 0.125, 0.5, 0.25, 0.0625, 0.020833]
    avg = 0.5*m + (1.0/36)*(9 + (-1)**(m + 1)) - (m/3.0 + 2.0/9)/2**m
    blocks = stringpart(binin, m)
    bign = len(blocks)
    lc = ([lincomplex(chunk) for chunk in blocks])
    t = ([-1.0*(((-1)**m)*(chunk-avg)+2.0/9) for chunk in lc])
    vg=np.histogram(t,bins=[-9999999999,-2.5,-1.5,-0.5,0.5,1.5,2.5,9999999999])[0][::-1]
    im=([((vg[ii]-bign*pi[ii])**2)/(bign*pi[ii]) for ii in range(7)])
    chisqr=functools.reduce(su,im)
    pval=spc.gammaincc(k/2.0,chisqr/2.0)
    return pval

In [28]:
linearcomplexitytest(ciphertext_bit*5,m=128)

0.54377912801395967

<h3>Randomness Test #11: Serial Test</h3>
Input: bit string, length of each block(m)

In [29]:
def serialtest(binin, m=16):
    ''' The focus of this test is the frequency of each and every overlapping m-bit pattern across the entire 
    sequence. The purpose of this test is to determine whether the number of occurrences of the 2m m-bit 
    overlapping patterns is approximately the same as would be expected for a random sequence. The pattern can 
    overlap.'''
    n = len(binin)
    hbin=binin+binin[0:m-1:]
    f1a = [hbin[xs:m+xs:] for xs in range(n)]
    oo=set(f1a)
    f1 = [f1a.count(xs)**2 for xs in oo]
    f1 = map(f1a.count,oo)
    cou=f1a.count
    f2a = [hbin[xs:m-1+xs:] for xs in range(n)]
    f2 = [f2a.count(xs)**2 for xs in set(f2a)]
    f3a = [hbin[xs:m-2+xs:] for xs in range(n)]
    f3 = [f3a.count(xs)**2 for xs in set(f3a)]
    psim1 = 0
    psim2 = 0
    psim3 = 0
    if m >= 0:
        suss = functools.reduce(su,f1)
        psim1 = 1.0 * 2 ** m * suss / n - n
    if m >= 1:
        suss = functools.reduce(su,f2)
        psim2 = 1.0 * 2 ** (m - 1) * suss / n - n
    if m >= 2:
        suss = functools.reduce(su,f3)
        psim3 = 1.0 * 2 ** (m - 2) * suss / n - n
    d1 = psim1-psim2
    d2 = psim1-2 * psim2 + psim3
    pval1 = spc.gammaincc(2 ** (m - 2), d1 / 2.0)
    pval2 = spc.gammaincc(2 ** (m - 3), d2 / 2.0)
    return [pval1, pval2]

In [30]:
serialtest(ciphertext_bit, m=16)

[0.49896108745922391, 0.49853075529672125]

<h3>Randomness Test #12: Approximate Entropy Test</h3>
Input: bit string, length of each block(m)

In [31]:
def aproximateentropytest(binin, m=10):
    ''' The focus of this test is the frequency of each and every overlapping m-bit pattern. The purpose of the 
    test is to compare the frequency of overlapping blocks of two consecutive/adjacent lengths (m and m+1) against 
    the expected result for a random sequence.'''
    n = len(binin)
    f1a = [(binin + binin[0:m - 1:])[xs:m + xs:] for xs in range(n)]
    f1 = [[xs, f1a.count(xs)] for xs in sorted(set(f1a))]
    f2a = [(binin + binin[0:m:])[xs:m + 1 + xs:] for xs in range(n)]
    f2 = [[xs, f2a.count(xs)] for xs in sorted(set(f2a))]
    c1 = [1.0 * f1[xs][1] / n for xs in range(len(f1))]
    c2 = [1.0 * f2[xs][1] / n for xs in range(len(f2))]
    phi1 = functools.reduce(su, list(map(logo, c1)))
    phi2 = functools.reduce(su, list(map(logo, c2)))
    apen = phi1 - phi2
    chisqr = 2.0 * n * (np.log(2) - apen)
    pval = spc.gammaincc(2 ** (m - 1), chisqr / 2.0)
    return pval

In [32]:
aproximateentropytest(ciphertext_bit*10, m=10)

1.2699250350691434e-35

<h3>Randomness Test #13: Cumulative Sums Test</h3>
Input: bit string
NOTE: Can be computed either forward or backward depending on mode chosen

In [33]:
def cumultativesumstest(binin):
    ''' The focus of this test is the maximal excursion (from zero) of the random walk defined by the cumulative 
    sum of adjusted (-1, +1) digits in the sequence. The purpose of the test is to determine whether the 
    cumulative sum of the partial sequences occurring in the tested sequence is too large or too small relative 
    to the expected behavior of that cumulative sum for random sequences.  This cumulative sum may be considered 
    as a random walk. For a random sequence, the random walk should be near zero. For non-random sequences, the 
    excursions of this random walk away from zero will be too large.'''
    n = len(binin)
    ss = [int(el) for el in binin]
    sc = list(map(sumi, ss))
    cs = np.cumsum(sc)
    z = max(abs(cs))
    ra = 0
    start = int(np.floor(0.25 * np.floor(-n / z) + 1))
    stop = int(np.floor(0.25 * np.floor(n / z) - 1))
    pv1 = []
    for k in range(start, stop + 1):
        pv1.append(sst.norm.cdf((4 * k + 1) * z / np.sqrt(n)) - sst.norm.cdf((4 * k - 1) * z / np.sqrt(n)))
    start = int(np.floor(0.25 * np.floor(-n / z - 3)))
    stop = int(np.floor(0.25 * np.floor(n / z) - 1))
    pv2 = []
    for k in range(start, stop + 1):
        pv2.append(sst.norm.cdf((4 * k + 3) * z / np.sqrt(n)) - sst.norm.cdf((4 * k + 1) * z / np.sqrt(n)))
    pval = 1
    pval -= functools.reduce(su, pv1)
    pval += functools.reduce(su, pv2)

    return pval

def cumultativesumstestreverse(binin):
    '''The focus of this test is the maximal excursion (from zero) of the random walk defined by the cumulative sum of adjusted (-1, +1) digits in the sequence. The purpose of the test is to determine whether the cumulative sum of the partial sequences occurring in the tested sequence is too large or too small relative to the expected behavior of that cumulative sum for random sequences.  This cumulative sum may be considered as a random walk. For a random sequence, the random walk should be near zero. For non-random sequences, the excursions of this random walk away from zero will be too large. '''
    pval=cumultativesumstest(binin[::-1])
    return pval

In [34]:
cumultativesumstest(ciphertext_bit)

0.81876977008479623

In [35]:
cumultativesumstestreverse(ciphertext_bit)

0.94926622884684653

<h3>Randomness Test #14: Random Excursions Test</h3>
Input: bit string

In [36]:
def pik(k,x):
    if k==0:
        out=1-1.0/(2*np.abs(x))
    elif k>=5:
        out=(1.0/(2*np.abs(x)))*(1-1.0/(2*np.abs(x)))**4
    else:
        out=(1.0/(4*x*x))*(1-1.0/(2*np.abs(x)))**(k-1)
    return out

def randomexcursionstest(binin):
    ''' The focus of this test is the number of cycles having exactly K visits in a cumulative sum random walk. 
    The cumulative sum random walk is found if partial sums of the (0,1) sequence are adjusted to (-1, +1). A 
    random excursion of a random walk consists of a sequence of n steps of unit length taken at random that begin 
    at and return to the origin. The purpose of this test is to determine if the number of visits to a state 
    within a random walk exceeds what one would expect for a random sequence.'''
    xvals=[-4, -3, -2, -1, 1, 2, 3, 4]
    ss = [int(el) for el in binin]
    sc = list(map(sumi,ss))
    cumsum = np.cumsum(sc)
    cumsum = np.append(cumsum,0)
    cumsum = np.append(0,cumsum)
    posi=np.where(cumsum==0)[0]
    cycles=([cumsum[posi[x]:posi[x+1]+1] for x in range(len(posi)-1)])
    j=len(cycles)
    sct=[]
    for ii in cycles:
        sct.append(([len(np.where(ii==xx)[0]) for xx in xvals]))
    sct=np.transpose(np.clip(sct,0,5))
    su=[]
    for ii in range(6):
        su.append([(xx==ii).sum() for xx in sct])
    su=np.transpose(su)
    pikt=([([pik(uu,xx) for uu in range(6)]) for xx in xvals])
    # chitab=1.0*((su-j*pikt)**2)/(j*pikt)
    chitab=np.sum(1.0*(np.array(su)-j*np.array(pikt))**2/(j*np.array(pikt)),axis=1)
    pval=([spc.gammaincc(2.5,cs/2.0) for cs in chitab])
    return pval

In [37]:
randomexcursionstest(ciphertext_bit)

[0.15477522218716219,
 0.069621934337598224,
 0.022694414426134263,
 0.066322893596628352,
 0.2955570040424837,
 0.72281309427843565,
 0.61834998263633456,
 0.93639478079058314]

<h3>Randomness Test #15: Random Excursions Variant Test</h3>
Input: bit string

In [38]:
def getfreq(linn, nu):
    val = 0
    for (x, y) in linn:
        if x == nu:
            val = y
    return val

def randomexcursionsvarianttest(binin):
    ''' The focus of this test is the number of times that a particular state occurs in a cumulative sum random walk. The purpose of this test is to detect deviations from the expected number of occurrences of various states in the random walk.'''
    ss = [int(el) for el in binin]
    sc = list(map(sumi, ss))
    cs = np.cumsum(sc)
    li = []
    for xs in sorted(set(cs)):
        if np.abs(xs) <= 9:
            li.append([xs, len(np.where(cs == xs)[0])])
    j = getfreq(li, 0) + 1
    pval = []
    for xs in range(-9, 9 + 1):
        if not xs == 0:
            # pval.append([xs, spc.erfc(np.abs(getfreq(li, xs) - j) / np.sqrt(2 * j * (4 * np.abs(xs) - 2)))])
            pval.append(spc.erfc(np.abs(getfreq(li, xs) - j) / np.sqrt(2 * j * (4 * np.abs(xs) - 2))))
    return pval

In [39]:
randomexcursionsvarianttest(ciphertext_bit)

[0.73160058895990132,
 0.67010315801033271,
 0.32679956766896601,
 0.52243128496156443,
 0.58233861910135132,
 0.59298009801742668,
 0.83302889371952138,
 0.58621368107313998,
 0.63735188823393707,
 0.098960154019405777,
 0.34080324688608477,
 0.3990751965482372,
 0.42267807417063552,
 0.47950012218695348,
 0.52243128496156443,
 0.55629846127473481,
 0.58388242077036523,
 0.60690542721795082]